In [1]:
import torch
import torch.nn as nn
from embedding import setup_vae, get_dataloader
from models.fc import MnistFullPredModel, MnistPredModel

In [2]:
config = {'alpha': 0., 'dset': 'mnist', 'obf_in': 25, 'obf_out': 15,
          'lip_reg': False, 'lip_coeff': 0.01, 'noise_reg': False,
          'sigma': 0.01, 'siamese_reg': False, 'margin': 25, 'lambda': 100.0}

In [3]:
dset = 'utkface'
vae, _ = setup_vae(dset=dset)
print("here")
# wts = torch.load("saved_models/fmnist_beta15_vae.pt")
wts = torch.load("saved_models/utkface_beta2_vae.pt")
print("loaded weights")
vae.load_state_dict(wts)

utkface
initialized VAE model
here
loaded weights


<All keys matched successfully>

In [4]:
loss_fn = torch.nn.CrossEntropyLoss()
def train(epoch, train_loader, pred_model, pred_optimizer):
        vae.eval()
        pred_model.train()
        train_loss = 0
        # Start training
        for batch_idx, (data, labels) in enumerate(train_loader):
            data, labels = data.cuda(), labels.cuda()
            # get sample embedding from the VAE
            with torch.no_grad():
                if dset in ['mnist', 'fmnist']:
                    data = data.view(-1, 784)
                mu, sigma = vae.encode(data)
                z = vae.reparametrize(mu, sigma)
                # z_tilde = vae.decode(z)
            z_tilde = z
            # z_tilde = data
            preds = pred_model(z_tilde.flatten(start_dim=1).detach())
            pred_loss = loss_fn(preds, labels)
            pred_optimizer.zero_grad()
            pred_loss.backward()
            pred_optimizer.step()
            
            train_loss += pred_loss.item()
        print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, pred_loss.item()))

In [5]:
def test(test_loader, pred_model):
    vae.eval()
    pred_model.eval()
    test_pred_loss = 0
    pred_correct = 0
    with torch.no_grad():
        for data, labels in test_loader:
            data, labels = data.cuda(), labels.cuda()
            if dset in ['mnist', 'fmnist']:
                data = data.view(-1, 784)
            mu, sigma = vae.encode(data)
            z = vae.reparametrize(mu, sigma)
            # z_tilde = vae.decode(z)
            z_tilde = z
            preds = pred_model(z_tilde.flatten(start_dim=1))
            pred_correct += (preds.argmax(dim=1) == labels).sum()
            # train obfuscator and pred_model
            pred_loss = loss_fn(preds, labels)
            test_pred_loss += pred_loss.item()

        test_pred_loss /= len(test_loader.dataset)
    pred_acc = pred_correct.item() / len(test_loader.dataset)
    print('====> Test pred loss: {:.4f}, acc {:.4f}'.format(test_pred_loss, pred_acc))

In [9]:
tr_loader, tst_loader = get_dataloader('utkface')
# model = MnistPredModel(25).cuda()
# model = MnistFullPredModel().cuda()
model = torch.nn.Sequential(nn.Linear(400, 500), nn.ReLU(), nn.Linear(500, 2)).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

In [11]:
for epoch in range(1, 200):
    # train(epoch, tr_loader, model, optimizer)
    test(tst_loader, model)

====> Test pred loss: 0.0047, acc 0.7031
====> Test pred loss: 0.0047, acc 0.6982


KeyboardInterrupt: 